In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x=train['Transported'],y=train['Age'])


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x="HomePlanet",hue="Transported",data=train)


In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='CryoSleep',hue='Transported',data=train)

We can conclude from the plot that people with crysleep is most likely to be transported

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x='Destination',hue='Transported',data=train)

In [ ]:
print('Missing values')
for value in train.columns:
    print(f'{value} : {train[value].isnull().sum()}')

In [ ]:
#Convert the Boolean Classes to Numerical
train.CryoSleep.replace({True:1,False:0},inplace=True)
train.VIP.replace({True:1,False:0},inplace=True)
train.Transported.replace({True:1,False:0},inplace=True)

In [ ]:
train.head()

In [ ]:
train['HomePlanet'].value_counts(dropna=False)

So if majority of passenger from Earth so replacing the NaN with Earth

In [ ]:
train['HomePlanet'].fillna('Earth',inplace=True)
train['HomePlanet'].value_counts(dropna=False)

In [ ]:
train['Destination'].value_counts(dropna=False)

In [ ]:
train['Destination'].fillna('TRAPPIST-1e',inplace=True)

train['HomePlanet'].value_counts(dropna=False)

In [ ]:
train["Cabin"].value_counts(dropna=False)


In [ ]:
train = train[train['Cabin'].notnull()]


In [ ]:
# Convert the Categorical Feature to Numerical Features

#custom binary encoding
train["Transported"] = np.where(train["Transported"]==True, 1, 0)

#Label Encoding
# Import label encoder
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
label_encoder_dest = preprocessing.LabelEncoder()
label_encoder_hp = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
train["Destination"] = label_encoder_dest.fit_transform(train["Destination"])
train["HomePlanet"] = label_encoder_hp.fit_transform(train["HomePlanet"])

In [ ]:
train.select_dtypes(include=float)


In [ ]:
#Age
plt.figure(figsize=(20,8))
plt.subplot(161)
sns.violinplot(data=train["Age"])

#Vrdeck
plt.subplot(162)
sns.violinplot(data=train['VRDeck'])

#roomservice
plt.subplot(163)
sns.violinplot(data=train['RoomService'])

#foodcourt
plt.subplot(164)
sns.violinplot(data=train['FoodCourt'])

#shoppingmall
plt.subplot(165)
sns.violinplot(data=train['ShoppingMall'])

In [ ]:
train[["Age","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]].describe()


In [ ]:
plt.figure(figsize=(20,8))

sns.histplot(train["Age"])


In [ ]:
"""
Note : 
1. As it can be seen the Age Group Bin where Max Passengers are present is in 18-32 Range with Mean as 28 and Median 27. 
2. Also it can be seen that for Variables such as "RoomService","FoodCourt","ShoppingMall","Spa","VRDeck" have median expenditures as 0, suggesting they dont spend much 
on the ammenties.
"""

train["Age"].fillna(27,inplace=True)
train["RoomService"].fillna(0,inplace=True)
train["FoodCourt"].fillna(0,inplace=True)
train["ShoppingMall"].fillna(0,inplace=True)
train["Spa"].fillna(0,inplace=True)
train["VRDeck"].fillna(0,inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
sns.countplot(x='CryoSleep',data=train)

In [ ]:
train['CryoSleep'].fillna(0.0,inplace=True)


In [ ]:
sns.countplot(x='VIP',data=train)

In [ ]:
train['VIP'].fillna(0.0,inplace=True)


In [ ]:
print('Missing values')
for value in train.columns:
    print(f'{value} : {train[value].isnull().sum()}')

In [ ]:
train=train[train['Name'].notnull()]

In [ ]:
train["Expenditure"]=train["RoomService"]+train["FoodCourt"]+train["ShoppingMall"]+train["Spa"]+train["VRDeck"]


In [ ]:
train.head()

In [ ]:
train.drop(columns=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],inplace=True)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(train.corr(),cmap="Greens",annot=True)
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler


In [ ]:
train["PassengerGroup"] = train["PassengerId"].apply(lambda x : x.split("_")[1])
# Binary Encoding 1 : Individual and 2 : Family
train["PassengerGroup"] = np.where(train["PassengerGroup"]=="01",1,2)

In [ ]:
min_max_Scaler = MinMaxScaler()
min_max_train=min_max_Scaler.fit(train[["PassengerGroup","HomePlanet","CryoSleep","Destination","Age","VIP","Expenditure"]])
vect_data=min_max_train.transform(train[["PassengerGroup","HomePlanet","CryoSleep","Destination","Age","VIP","Expenditure"]])


In [ ]:
vect_train=pd.DataFrame(columns=["PassengerGroup","HomePlanet","CryoSleep","Destination","Age","VIP","Expenditure"],data=vect_data)
vect_train["Transported"] = train["Transported"]
vect_train

In [ ]:
vect_train.to_csv("vectorized_train.csv",index=False)


**Model**

Logistic regression

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X=vect_train.drop('Transported',axis=1)
y=train['Transported']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
X_train.isnull().sum()


In [ ]:
print(X.shape , y.shape)


In [ ]:
"""
Model 1 : Logistic Regression (Default Regularization : l2 Penalty {Ridge})
"""
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix , accuracy_score , f1_score , precision_score , recall_score

In [ ]:
LR = LogisticRegression()
LR.fit(X_train,y_train)

In [ ]:
y_train_pred = LR.predict(X_train)
print("confusion matrix : " ,confusion_matrix(y_train, y_train_pred))
print("accuracy : ",round(accuracy_score(y_train, y_train_pred),2))
print("f1-score : ",round(f1_score(y_train, y_train_pred),2))
print("precision : ",round(precision_score(y_train, y_train_pred),2))
print("recall : " ,round(recall_score(y_train, y_train_pred),2))

In [ ]:
y_pred = LR.predict(X_test)
y_proba = LR.predict_proba(X_test)

In [ ]:
print("confusion matrix : " ,confusion_matrix(y_test, y_pred))
print("accuracy : ",round(accuracy_score(y_test, y_pred),2))
print("f1-score : ",round(f1_score(y_test, y_pred),2))
print("precision : ",round(precision_score(y_test, y_pred),2))
print("recall : " ,round(recall_score(y_test, y_pred),2))


In [ ]:
"""
Model 2 : Logistic Regression (L1 Norm/Penalty {Lasso})
"""
lr = LogisticRegression(penalty="l1",solver="liblinear")
lr.fit(X_train,y_train)

In [ ]:
y_train_pred = lr.predict(X_train)
print("confusion matrix : " ,confusion_matrix(y_train, y_train_pred))
print("accuracy : ",round(accuracy_score(y_train, y_train_pred),2))
print("f1-score : ",round(f1_score(y_train, y_train_pred),2))
print("precision : ",round(precision_score(y_train, y_train_pred),2))
print("recall : " ,round(recall_score(y_train, y_train_pred),2))

In [ ]:
y_pred = lr.predict(X_test)
y_proba = lr.predict_proba(X_test)

In [ ]:
print("confusion matrix : " ,confusion_matrix(y_test, y_pred))
print("accuracy : ",round(accuracy_score(y_test, y_pred),2))
print("f1-score : ",round(f1_score(y_test, y_pred),2))
print("precision : ",round(precision_score(y_test, y_pred),2))
print("recall : " ,round(recall_score(y_test, y_pred),2))

In [ ]:
print("Model the Beta Coefficients:\n")
print("Beta0 :", lr.intercept_[0])
  
for x,beta in enumerate(lr.coef_[0]):
    print(f"Beta{x}/{lr.feature_names_in_[x]} : {round(beta,2)}")

**Submit Test**

In [ ]:
submit_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
print('Missing values in submit test')
for value in submit_test.columns:
    print(f'{value}:{submit_test[value].isnull().sum()}')
    

In [ ]:
submit_test["PassengerGroup"] = submit_test["PassengerId"].apply(lambda x : x.split("_")[1])
submit_test["PassengerGroup"] = np.where(submit_test["PassengerGroup"]=="01",1,2)

submit_test["HomePlanet"].fillna("Earth",inplace=True)
submit_test["Destination"].fillna("TRAPPIST-1e",inplace=True)
submit_test.drop('Cabin',inplace = True,axis=1)

submit_test.CryoSleep.fillna(0,inplace=True)
submit_test.VIP.fillna(0,inplace=True)
submit_test.CryoSleep.replace({True:1,False:0},inplace=True)
submit_test.VIP.replace({True:1,False:0},inplace=True)

submit_test["Age"].fillna(27,inplace=True)
submit_test["RoomService"].fillna(0,inplace=True)
submit_test["FoodCourt"].fillna(0,inplace=True)
submit_test["ShoppingMall"].fillna(0,inplace=True)
submit_test["Spa"].fillna(0,inplace=True)
submit_test["VRDeck"].fillna(0,inplace=True)
submit_test["Expenditure"]=submit_test["RoomService"]+submit_test["FoodCourt"]+submit_test["ShoppingMall"]+submit_test["Spa"]+submit_test["VRDeck"]

submit_test["Destination"] = label_encoder_dest.fit_transform(submit_test["Destination"])
submit_test["HomePlanet"] = label_encoder_hp.fit_transform(submit_test["HomePlanet"])

submit_test = submit_test[["PassengerGroup","HomePlanet","CryoSleep","Destination","Age","VIP","Expenditure"]]
submit_test = min_max_train.transform(submit_test[["PassengerGroup","HomePlanet","CryoSleep","Destination","Age","VIP","Expenditure"]])

In [ ]:
submit_test = pd.DataFrame(data = submit_test, columns =X_train.columns)#.drop(columns=["Destination","Age","VIP","Cabin_side"])


In [ ]:
y_submit_pred = lr.predict(submit_test)


In [ ]:
submit=pd.DataFrame(data=y_submit_pred,columns=["Transported"])
submit_test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submit["PassengerId"]=submit_test["PassengerId"]
submit["Transported"].replace({1:True,0:False},inplace=True)

In [ ]:
submit.to_csv("submission2.csv",index=False)

In [ ]:
submit